In [ ]:
# INSTALACOES #

# !pip install --upgrade pip
%pip install yfinance
%pip install crewai
%pip install "crewai[tools]"
%pip install langchain
%pip install langchain-openai
%pip install langchain-community
%pip install duckduckgo
%pip install -U duckduckgo-search

In [ ]:
# IMPORTACOES #
import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown


In [ ]:
# Exemplo bobo
#yf.download(ticket,start,end)
#stock = yf.download('AAPL', start='2023-08-08', end='2024-08-08')

# funcao
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start='2023-08-08', end='2024-08-08')
    return stock

# Criando Yahoo Fincance Tool
yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetch stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
    func= lambda ticket: fetch_stock_price(ticket)
)


In [ ]:
# Testando tool
response = yahoo_finance_tool('AAPL')
print(response)

In [ ]:
# Importando OpenAI LLM - GPT
os.environ['OPENAI_API_KEY'] = ''
llm = ChatOpenAI(model='gpt-3.5-turbo')

In [ ]:
stockPriceAnalyst = Agent (
    role = 'Senior Stock Analyst',
    goal = 'Find the {ticket} stock price and analyses trends',
    backstory = 'You are a highly experienced in analyzing the price of an specific stock and make predictions about its future price.',
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    tools = [yahoo_finance_tool],
    allow_delegation = False
)

In [ ]:
getStockPrice = Task(
    description = 'Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways',
    expected_output = "Specify the current trend stock price - up, down or sideaways, eg. stock='APPL, price UP'",
    agent = stockPriceAnalyst
)

In [ ]:
# IMPORTANDO A TOOL DE PESQUISA
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [ ]:
newsAnalyst = Agent (
    role = 'Stock News Analyst',
    goal = """Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with the news context.
              For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extrene greeed.""",
    backstory = """You are a highly experienced in analyzing the market news and have tracked assets for more than 10 years.
                   You are also master level analyts in the traditional markets and have deep understanding of human psychology.
                   You understand news, theirs tittles and informations, but you look at those with a health dose of skepticism.
                   You consider also the source of the news articles.""",
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    tools = [search_tool],
    allow_delegation = False
)

In [ ]:
getNews = Task(
    description = f"""Take the stock and always include BTC to it (if not request).
                      Use the search tool to search each one individualy.
                      The current date is {datetime.now()}
                      Compose the results into a helpfull report.""",
    expected_output = """A summary of the overall market and one sentece summary for each request asset.
                         Include a fear/greed score for each asset based on the news. Use format
                         <STOCK ASSET>
                         <SUMMARY BASED ON THE NEWS>
                         <TREND PREDICTION>
                         <FEAR/GREED SCORE>""",
    agent = newsAnalyst
)

In [ ]:
stockAnalystWrite = Agent(
    role = 'Senior Stock Analyst Writter',
    goal = 'Analyze the trend price and news and write insightfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend.',
    backstory = """You are widely accepted as the best stock analyst in the market.
                   You understand complex concepts and create compelling stories and narratives that resonate with wider audiences.
                   You understand macro factors and combine multiple theories. eg. cycle theory and fundamental analyses.
                   You are able to hold multiple opinions when analyzing anything""",
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    allow_delegation = True
)

In [ ]:
writeAnalyses = Task(
    description = """Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
                     that is brief and highlights the most important points.
                     Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
                     Include the previous analyses of stock trend and news summary.""",
    expected_output = """An elouquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contains:
                         - 3 bullet executive summary
                         - Introduction - set the overall picture and spike up the interest
                         - Main part provides the meat of the analysis including the news summary and fear/greed scores
                         - Summary - key facts and concrete future trend prediction - up, down or sideways""",
    agent = stockAnalystWrite,
    context = [getStockPrice, getNews]
)

In [ ]:
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
    tasks = [getStockPrice, getNews, writeAnalyses],
    verbose = True, #verbose = 2,
    process = Process.hierarchical,
    full_output = True,
    share_crew = False,
    manager_llm = llm,
    max_iter = 15
)

In [ ]:
results = crew.kickoff(inputs={'ticket':'AAPL'})

In [ ]:
list(results.keys())

In [ ]:
results['final_output']

In [ ]:
Markdown(results['final_output'])